# Customized RNN
## Brief
TensorFlow API r1.3

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
class MyRnnCell(tf.nn.rnn_cell.RNNCell):
    def __init__(self, state_size, dtype):
        self._state_size = state_size
        self._dtype = dtype

    def __call__(self, _input, state):
        self._W_xh = tf.get_variable(shape=[self._state_size, self._state_size], dtype=self._dtype, name="W_xh")
        self._W_hh = tf.get_variable(shape=[self._state_size, self._state_size], dtype=self._dtype, name="W_hh")
        self._W_ho = tf.get_variable(shape=[self._state_size, self._state_size], dtype=self._dtype, name="W_ho")
        self._b_h = tf.get_variable(shape=[self._state_size], dtype=self._dtype, name="b_h")
        self._b_o = tf.get_variable(shape=[self._state_size], dtype=self._dtype, name="b_o")
        new_state = tf.tanh(tf.matmul(_input,self._W_xh)+tf.matmul(state, self._W_hh))
        new_output = tf.tanh(tf.matmul(new_state, self._W_ho+self._b_o))
        return new_output, new_state
        

In [ ]:
test_cell = MyRnnCell(100, tf.float64)

In [ ]:
tf.reset_default_graph()
x=np.zeros([1,100])
state = np.zeros([1,100])
with tf.Session() as sess:
    pass